### Data Processing 
#### Takes "time_series_coid19_confirmed_US.csv" and does the following processing steps:

 - Reads in cvs file as Pandas dataframe 
 - Renames "Long_" factor to "Lon" for clarity 
 - Drops any rows with missing data: undefined regions, and regions outside the 50 states (removing overseas US territories)
 - Drops rows contating and Null or NaN feilds 
 - Saves cleaned dataframe as "Covid_Data_Cleaned_All_States.csv" to the Data directory 
 - Runs a function [buildStateCSV] that outputs the following:
     - 52 Unique data frame for each US State (including DC and Puerto Rico
     - Each data frame drops all extraneous feilds, leaving only Countys within the respective US state
     - Dataframe is transposed so that the date feild is a vertical column for time series analysis
     - Saves each dataframe as a csv file "StatesName.csv" in the Data/States directory 
     

In [2]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
cd ..

/Users/sreidy/Documents/UMBC 2020/fall2020/DATA601/HW1


In [4]:
#Load in the csv File 
covid_df = pd.read_csv("Data/time_series_covid19_confirmed_US.csv")
covid_df

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,9/7/20,9/8/20,9/9/20,9/10/20,9/11/20,9/12/20,9/13/20,9/14/20,9/15/20,9/16/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1383,1385,1398,1413,1420,1432,1442,1447,1585,1619
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,4586,4609,4639,4678,4722,4752,4781,4800,4978,5003
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,617,617,618,618,618,620,622,626,801,809
3,84001007,US,USA,840,1007.0,Bibb,Alabama,US,32.996421,-87.125115,...,557,562,564,566,574,576,578,581,610,612
4,84001009,US,USA,840,1009.0,Blount,Alabama,US,33.982109,-86.567906,...,1070,1071,1082,1099,1109,1114,1121,1128,1464,1487
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3335,84056039,US,USA,840,56039.0,Teton,Wyoming,US,43.935225,-110.589080,...,447,450,451,453,462,465,470,476,478,481
3336,84056041,US,USA,840,56041.0,Uinta,Wyoming,US,41.287818,-110.547578,...,308,309,308,310,310,310,311,312,312,323
3337,84090056,US,USA,840,90056.0,Unassigned,Wyoming,US,0.000000,0.000000,...,0,0,0,0,0,0,2,2,2,0
3338,84056043,US,USA,840,56043.0,Washakie,Wyoming,US,43.904516,-107.680187,...,111,111,111,111,111,111,111,111,111,111


In [5]:
#info on the datatypes (too many colls to display)
covid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3340 entries, 0 to 3339
Columns: 250 entries, UID to 9/16/20
dtypes: float64(3), int64(241), object(6)
memory usage: 6.4+ MB


In [6]:
#rename "Long_" to Lon for simplisity 
covid_df = covid_df.rename(columns={'Long_': 'Lon'})

In [7]:
#removing undefined regions, and regions outside the 50 states (removing overseas US territories) 
covid_df[covid_df['Admin2'].isna()]

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Lon,...,9/7/20,9/8/20,9/9/20,9/10/20,9/11/20,9/12/20,9/13/20,9/14/20,9/15/20,9/16/20
100,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
336,84088888,US,USA,840,88888.0,NaN,Diamond Princess,US,0.0000,0.0000,...,49,49,49,49,49,49,49,49,49,49
570,84099999,US,USA,840,99999.0,NaN,Grand Princess,US,0.0000,0.0000,...,103,103,103,103,103,103,103,103,103,103
571,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,1619,1713,1758,1846,1863,1863,1863,1927,1966,2013
2121,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,59,59,59,59,59,60,60,61,61,61
3007,850,VI,VIR,850,78.0,NaN,Virgin Islands,US,18.3358,-64.8963,...,1190,1191,1197,1201,1201,1211,1220,1221,1232,1232


In [8]:
#removing NaN values 
covid_df_clean = covid_df.dropna(axis = 'rows')
covid_df_clean.shape

(3324, 250)

In [9]:
#checking for undefined regions within the US with no defnined Lat / Lon
bool_mask = covid_df_clean["Lat"] == 0
covid_df_clean[bool_mask]

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Lon,...,9/7/20,9/8/20,9/9/20,9/10/20,9/11/20,9/12/20,9/13/20,9/14/20,9/15/20,9/16/20
52,84080001,US,USA,840,80001.0,Out of AL,Alabama,US,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
64,84090001,US,USA,840,90001.0,Unassigned,Alabama,US,0.0,0.0,...,11499,11727,11837,12298,12606,12823,12960,13160,0,0
89,84080002,US,USA,840,80002.0,Out of AK,Alaska,US,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
95,84090002,US,USA,840,90002.0,Unassigned,Alaska,US,0.0,0.0,...,1,1,1,1,1,1,1,2,2,1
111,84080004,US,USA,840,80004.0,Out of AZ,Arizona,US,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3233,84090054,US,USA,840,90054.0,Unassigned,West Virginia,US,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3285,84080055,US,USA,840,80055.0,Out of WI,Wisconsin,US,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3304,84090055,US,USA,840,90055.0,Unassigned,Wisconsin,US,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3329,84080056,US,USA,840,80056.0,Out of WY,Wyoming,US,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
#if Lat and Lon saved as 0.0 its an undefined location and its to be removed 
covid_df_clean = covid_df_clean[~(covid_df_clean.Lat == 0)]
covid_df_clean.shape

(3220, 250)

#### Building A CSV for Each State 

In [11]:
cd Data

/Users/sreidy/Documents/UMBC 2020/fall2020/DATA601/HW1/Data


In [19]:
#save data to csv
covid_df_clean.to_csv("Covid_Data_Cleaned_All_States.csv")

In [12]:
#get list of states in data 
states = covid_df_clean.Province_State.unique()

In [13]:
cd States

/Users/sreidy/Documents/UMBC 2020/fall2020/DATA601/HW1/Data/States


In [14]:
def buildStateCSV(df,states_list):
    
    for state in states_list:
        bool_mask = df["Province_State"] == state
        segdf = df[bool_mask]
        #get list of counties in each state 
        county = segdf.Admin2.unique()
        #transpose the dataframe where dates are all within one col 
        tdf = segdf.T
        #resize dataframe to only incudle county, date, and case numbers
        temp = tdf.iloc[11:]
        #rename col to county names 
        temp.columns = county
        
        
        temp.index.name = 'Date'
        temp.reset_index()
        
        
    
        #print(temp.columns)
        cols = list(temp)
        #temp["Total"] = temp[cols[1:]].sum(axis=1)
        temp["Total"] = temp.sum(axis=1)
      
        temp.to_csv(state + ".csv")
        print (state + " saved")
      
        

In [15]:
buildStateCSV(covid_df_clean, states)

<ipython-input-14-25a8b9e017aa>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp["Total"] = temp.sum(axis=1)


Alabama saved
Alaska saved
Arizona saved
Arkansas saved
California saved
Colorado saved
Connecticut saved
Delaware saved
District of Columbia saved
Florida saved
Georgia saved
Hawaii saved
Idaho saved
Illinois saved
Indiana saved
Iowa saved
Kansas saved
Kentucky saved
Louisiana saved
Maine saved
Maryland saved
Massachusetts saved
Michigan saved
Minnesota saved
Mississippi saved
Missouri saved
Montana saved
Nebraska saved
Nevada saved
New Hampshire saved
New Jersey saved
New Mexico saved
New York saved
North Carolina saved
North Dakota saved
Ohio saved
Oklahoma saved
Oregon saved
Pennsylvania saved
Puerto Rico saved
Rhode Island saved
South Carolina saved
South Dakota saved
Tennessee saved
Texas saved
Utah saved
Vermont saved
Virginia saved
Washington saved
West Virginia saved
Wisconsin saved
Wyoming saved


In [133]:
#Example of Marland Data
df = pd.read_csv("Maryland.csv")
df.dtypes

Date                object
Allegany             int64
Anne Arundel         int64
Baltimore            int64
Baltimore City       int64
Calvert              int64
Caroline             int64
Carroll              int64
Cecil                int64
Charles              int64
Dorchester           int64
Frederick            int64
Garrett              int64
Harford              int64
Howard               int64
Kent                 int64
Montgomery           int64
Prince George's      int64
Queen Anne's         int64
Somerset             int64
St. Mary's           int64
Talbot               int64
Washington           int64
Wicomico             int64
Worcester            int64
Total              float64
dtype: object

In [129]:
#Set Date as a pandas datetime64[ns] 
df['Date']  = pd.to_datetime(df['Date'], format='%m/%d/%y')
df = df.set_index('Date')
df.index


DatetimeIndex(['2020-01-22', '2020-01-23', '2020-01-24', '2020-01-25',
               '2020-01-26', '2020-01-27', '2020-01-28', '2020-01-29',
               '2020-01-30', '2020-01-31',
               ...
               '2020-09-07', '2020-09-08', '2020-09-09', '2020-09-10',
               '2020-09-11', '2020-09-12', '2020-09-13', '2020-09-14',
               '2020-09-15', '2020-09-16'],
              dtype='datetime64[ns]', name='Date', length=239, freq=None)

In [130]:
df

,Allegany,Anne Arundel,Baltimore,Baltimore City,Calvert,Caroline,Carroll,Cecil,Charles,Dorchester,...,Montgomery,Prince George's,Queen Anne's,Somerset,St. Mary's,Talbot,Washington,Wicomico,Worcester,Total
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-22,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
2020-01-23,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
2020-01-24,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
2020-01-25,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
2020-01-26,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-12,415,9065,16648,15183,854,606,1842,904,2554,514,...,21089,27589,606,216,1186,508,1474,1754,930,115533.0
2020-09-13,421,9119,16724,15254,862,609,1852,911,2570,527,...,21167,27683,607,219,1193,510,1493,1770,940,116110.0
2020-09-14,424,9191,16797,15299,874,609,1863,918,2591,530,...,21281,27760,612,220,1197,512,1506,1778,948,116646.0


In [ ]:
###